# Symbolic neural networks for cognitive capacities
[Paper](http://reason.cs.uiuc.edu/tsvi/BICA_93_Main.pdf)

## Example of a more complex symbolic recognition and recall

Take 3 men (Adam, Baker, Charlie) and 3 women (Diane, Edna, Fay).

Make a matrix of features. Gender (1=male, 0=female), A? (1=yes, 0=no), B?, ... etc

In [1]:
import numpy as np
exp = np.matrix('1, 1, 0, 0, 0, 0, 0; 1, 0, 1, 0, 0, 0, 0; 1, 0, 0, 1, 0, 0, 0; 0, 0, 0, 0, 1, 0, 0; 0, 0, 0, 0, 0, 1, 0; 0, 0, 0, 0, 0, 0, 1') # m/f, able, baker, charlie, diane, edna, fay
print(exp)
print(exp.shape)

[[1 1 0 0 0 0 0]
 [1 0 1 0 0 0 0]
 [1 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1]]
(6, 7)


Check that each instance is recognised

In [2]:
pinv = np.linalg.pinv(exp)
pinv_transpose = pinv.transpose()

for index in range(exp.shape[0]):
    prediction = pinv_transpose * exp[index, :].T
    print(index + 1, round(prediction.item( (index, 0)), 2))

1 1.0
2 1.0
3 1.0
4 1.0
5 1.0
6 1.0


# Categories

Define two categories, male and female.  These are the SUM of the observations in each set.

In [3]:
male = np.matrix(np.zeros(exp.shape[1]))
female = np.matrix(np.zeros(exp.shape[1]))
for index in range(exp.shape[0]):
    if exp.item( (index, 0) ) == 1:
        male += exp[index, :]
    else:
        female += exp[index, :]
print("male  ", male)
print("female", female)

male   [[ 3.  1.  1.  1.  0.  0.  0.]]
female [[ 0.  0.  0.  0.  1.  1.  1.]]


Check predictions of each category

In [4]:
prediction_male = pinv_transpose * male.T
prediction_male

matrix([[ 1.],
        [ 1.],
        [ 1.],
        [ 0.],
        [ 0.],
        [ 0.]])

In [5]:
prediction_female = pinv_transpose * female.T
prediction_female

matrix([[ 0.],
        [ 0.],
        [ 0.],
        [ 1.],
        [ 1.],
        [ 1.]])

Explore confounding cases

In [6]:
females_with_male_names = np.matrix('0; 1; 1; 1; 0; 0; 0')
prediction_male_names = pinv_transpose * females_with_male_names
prediction_male_names

matrix([[ 0.25],
        [ 0.25],
        [ 0.25],
        [ 0.  ],
        [ 0.  ],
        [ 0.  ]])

In [7]:
males_with_female_names = np.matrix('1; 0; 0; 0; 1; 1; 1')
prediction_female_names = pinv_transpose * males_with_female_names
prediction_female_names

matrix([[ 0.25],
        [ 0.25],
        [ 0.25],
        [ 1.  ],
        [ 1.  ],
        [ 1.  ]])

## Add a new person, George

In [8]:
new_exp = np.concatenate((exp, np.matrix('0; 0; 0; 0; 0; 0')), axis=1)
print(new_exp)
george = np.matrix('1, 0, 0, 0, 0, 0, 0, 1')
new_exp = np.concatenate((new_exp, george), axis=0)
new_exp

[[1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0]
 [1 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]]


matrix([[1, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0, 0, 0, 1]])

Check that George is recognised

In [9]:
pinv = np.linalg.pinv(new_exp)
pinv_transpose = pinv.transpose()
predict_george = pinv_transpose * george.transpose()
predict_george.round()

matrix([[ 0.],
        [-0.],
        [-0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 1.]])

Is George male?

In [10]:
george.item( (0,0) )

1

## Add _female_ George (like [George Eliot](https://en.wikipedia.org/wiki/George_Eliot))

In [11]:
# Add new column
new_column = np.matrix(np.zeros(new_exp.shape[0]))
new_column = new_column.T
print(new_column)
new_exp2 = np.concatenate((new_exp, new_column), axis=1)
print(new_exp2)
new_row = [0 for item in range(new_exp.shape[1])]
new_row.append(1)
george_f = np.matrix(new_row)
print(george_f)
new_exp2 = np.concatenate((new_exp2, george_f), axis=0)
new_exp2

[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
[[ 1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  1.  0.]]
[[0 0 0 0 0 0 0 0 1]]


matrix([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

Check that female George is recognised

In [12]:
pinv = np.linalg.pinv(new_exp2)
pinv_transpose = pinv.transpose()
predict_george_f = pinv_transpose * george_f.transpose()
predict_george_f.round()

matrix([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 1.]])

Is George_f male?

In [13]:
george_f.item( (0,0) )

0

Update to categories

In [14]:
male = np.matrix('4; 1; 1; 1; 0; 0; 0; 1; 0')
prediction_male = pinv_transpose * male
prediction_male.round(2)

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.]])

In [15]:
female = np.matrix('0; 0; 0; 0; 1; 1; 1; 0; 1')
prediction_female = pinv_transpose * female
prediction_female.round(2)

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.]])

In [16]:
georges = np.matrix('1; 0; 0; 0; 0; 0; 0; 1; 1')
prediction_georges = pinv_transpose * georges
prediction_georges.round(2)

array([[ 0.],
       [-0.],
       [-0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.]])